In [ ]:
import os
from google.cloud import bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\path\your-google-analytics-key.json"

client = bigquery.Client()

The dataset 'bigquery-public-data.google_analytics_sample.ga_sessions_*' from **Google Analytics Sample** as a google cloud public data has the following columns:

- visitorId
- visitNumber
- visitId
- visitStartTime
- date
- totals
- trafficSource
- device
- geoNetwork
- customDimensions
- hits
- fullVisitorId
- userId
- channelGrouping
- socialEngagement


In [2]:
# Extract count of visits in each channel grouping in 2017 July
query = """
SELECT
  channelGrouping,
  COUNT(visitId) AS number_of_visits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY channelGrouping
HAVING number_of_visits > 0
ORDER BY number_of_visits DESC;
"""

# Execute the query and store the results in a Pandas DataFrame
df = client.query(query).to_dataframe()

# Display the first few rows
df.head()

,channelGrouping,number_of_visits
0,Organic Search,37655
1,Direct,12306
2,Referral,9518
3,Social,7749
4,Paid Search,2105


# Data Structure

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Extract count of visits in each channel grouping from a sample date
query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170715`;
"""

# Execute the query and store the results in a Pandas DataFrame
ex = client.query(query).to_dataframe()

# Display the first few rows
ex.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1500175678,1500175678,20170715,"{'visits': 1, 'hits': 15, 'pageviews': 11, 'ti...","{'referralPath': None, 'campaign': 'Data Share...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Easte...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 20, 'minu...",8150256204624206878,None,None,Affiliates,Not Socially Engaged
1,<NA>,1,1500140302,1500140302,20170715,"{'visits': 1, 'hits': 15, 'pageviews': 9, 'tim...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu...",5465216125812676280,None,None,Organic Search,Not Socially Engaged
2,<NA>,1,1500153151,1500153151,20170715,"{'visits': 1, 'hits': 15, 'pageviews': 14, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 14, 'minu...",394683673490644764,None,None,Organic Search,Not Socially Engaged
3,<NA>,1,1500103046,1500103046,20170715,"{'visits': 1, 'hits': 15, 'pageviews': 15, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Opera Mini', 'browserVersion': 'n...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 0, 'minut...",0754507752413433588,None,None,Organic Search,Not Socially Engaged
4,<NA>,1,1500157730,1500157730,20170715,"{'visits': 1, 'hits': 16, 'pageviews': 15, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",3134978009959363466,None,None,Organic Search,Not Socially Engaged


In [5]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           1721 non-null   Int64 
 2   visitId               1721 non-null   Int64 
 3   visitStartTime        1721 non-null   Int64 
 4   date                  1721 non-null   object
 5   totals                1721 non-null   object
 6   trafficSource         1721 non-null   object
 7   device                1721 non-null   object
 8   geoNetwork            1721 non-null   object
 9   customDimensions      1721 non-null   object
 10  hits                  1721 non-null   object
 11  fullVisitorId         1721 non-null   object
 12  userId                0 non-null      object
 13  clientId              0 non-null      object
 14  channelGrouping       1721 non-null   object
 15  socialEngagementType  1721 non-null   

In [52]:
ex['channelGrouping'].unique()

array(['Affiliates', 'Organic Search', 'Referral', 'Social', 'Direct',
       'Paid Search', 'Display'], dtype=object)

visitorId, userId, and clientId provide no useful information.

In [6]:
for col in ex.select_dtypes(object):
    try:
        ex[f'{col}_info'] = [x.keys() for x in ex[col]]
        print(ex[f'{col}_info'].value_counts())
        print("\n")
    except AttributeError:
        continue

totals_info
(visits, hits, pageviews, timeOnSite, bounces, transactions, transactionRevenue, newVisits, screenviews, uniqueScreenviews, timeOnScreen, totalTransactionRevenue, sessionQualityDim)    1721
Name: count, dtype: int64


trafficSource_info
(referralPath, campaign, source, medium, keyword, adContent, adwordsClickInfo, isTrueDirect, campaignCode)    1721
Name: count, dtype: int64


device_info
(browser, browserVersion, browserSize, operatingSystem, operatingSystemVersion, isMobile, mobileDeviceBranding, mobileDeviceModel, mobileInputSelector, mobileDeviceInfo, mobileDeviceMarketingName, flashVersion, javaEnabled, language, screenColors, screenResolution, deviceCategory)    1721
Name: count, dtype: int64


geoNetwork_info
(continent, subContinent, country, region, metro, city, cityId, networkDomain, latitude, longitude, networkLocation)    1721
Name: count, dtype: int64




In [18]:
pd.DataFrame(ex['totals'].tolist()).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721 entries, 0 to 1720
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   visits                   1721 non-null   int64  
 1   hits                     1721 non-null   int64  
 2   pageviews                1721 non-null   int64  
 3   timeOnSite               752 non-null    float64
 4   bounces                  966 non-null    float64
 5   transactions             16 non-null     float64
 6   transactionRevenue       16 non-null     float64
 7   newVisits                1407 non-null   float64
 8   screenviews              0 non-null      object 
 9   uniqueScreenviews        0 non-null      object 
 10  timeOnScreen             0 non-null      object 
 11  totalTransactionRevenue  16 non-null     float64
 12  sessionQualityDim        1721 non-null   int64  
dtypes: float64(6), int64(4), object(3)
memory usage: 174.9+ KB


In [38]:
pd.DataFrame(ex['trafficSource'].tolist()).sample(5)

,referralPath,campaign,source,medium,keyword,adContent,adwordsClickInfo,isTrueDirect,campaignCode
271,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
833,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
292,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
1306,None,(not set),google,organic,(not provided),None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None
222,None,Data Share Promo,Partners,affiliate,None,None,"{'campaignId': None, 'adGroupId': None, 'creat...",None,None


In [9]:
pd.DataFrame(ex['device'].tolist()).head()

,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,javaEnabled,language,screenColors,screenResolution,deviceCategory
0,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
1,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
2,Safari,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
3,Opera Mini,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
4,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [10]:
pd.DataFrame(ex['geoNetwork'].tolist()).head()

,continent,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Europe,Eastern Europe,Czechia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,iol.cz,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,cox.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Americas,Northern America,Canada,Quebec,(not set),Montreal,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Asia,Southern Asia,India,Karnataka,(not set),Bengaluru,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Palo Alto,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset


# Traffic and Conversion Analysis

Focus on visitors that has converted once or none from 2016-August to 2017-July

In [58]:
traffic_conversion_query = """
WITH conversions_summed AS (
    SELECT 
        fullVisitorId, 
        visitNumber, 
        date,
        channelGrouping,
        totals.transactions AS transactions, 
        totals.totalTransactionRevenue / 1000000 AS value,
        totals.newVisits AS newVisitor,
        trafficSource.source AS source,
        trafficSource.medium AS medium,
        trafficSource.isTrueDirect AS isTrueDirect,

        -- Conversion indicator
        (CASE WHEN totals.transactions IS NULL THEN 0 ELSE 1 END) AS conversion,
        
        -- Running total of conversions for each visitor
        SUM(CASE WHEN totals.transactions IS NULL THEN 0 ELSE 1 END) 
        OVER(
            PARTITION BY fullVisitorId
            ORDER BY visitNumber
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW    
        ) AS convSoFar
     
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
),

aggregated_features AS (
    SELECT 
        fullVisitorId, 
        MIN(date) AS first_visit_date, 
        MAX(newVisitor) AS isNewVisit,  -- Ensures that if a visitor was new at least once, we mark them as new
        MIN(CASE WHEN conversion = 1 THEN date ELSE NULL END) AS first_conversion_date  -- Find the first conversion date for each visitor
    FROM conversions_summed
    GROUP BY fullVisitorId
)

SELECT 
    c.fullVisitorId, 
    c.date, 
    c.conversion, 
    c.convSoFar,
    c.value, 
    c.source, 
    c.medium, 
    c.channelGrouping,
    c.isTrueDirect,
    c.newVisitor,
    
    -- Join aggregated features
    af.isNewVisit,
    CASE 
        WHEN c.date = af.first_conversion_date THEN 1 
        ELSE 0 
    END AS isFirstConversion

FROM conversions_summed c
LEFT JOIN aggregated_features af 
    ON c.fullVisitorId = af.fullVisitorId;
"""

In [59]:
# Execute the query and store the results in a Pandas DataFrame
traffic_df = client.query(traffic_conversion_query).to_dataframe()

# Display the first few rows
traffic_df.head()

,fullVisitorId,date,conversion,convSoFar,value,source,medium,channelGrouping,isTrueDirect,newVisitor,isNewVisit,isFirstConversion
0,0001235242196175467,20161101,0,0,NaN,baidu,organic,Organic Search,<NA>,1,1,0
1,0015980521034237800,20161208,0,0,NaN,facebook.com,referral,Social,<NA>,1,1,0
2,0034572692224994048,20161003,0,0,NaN,baidu,organic,Organic Search,<NA>,1,1,0
3,0040335064426021006,20160831,0,0,NaN,phandroid.com,referral,Referral,<NA>,1,1,0
4,0047788902811115718,20161214,0,0,NaN,adwords.google.com,referral,Referral,<NA>,1,1,0


In [60]:
# Turn 'date' to datetime object
traffic_df['date'] = pd.to_datetime(traffic_df['date'], format = "%Y%m%d")

In [61]:
# Save to CSV in a specific directory
traffic_df.to_csv("agg_data/conv_traf.csv", index=False)

# Google and Youtube Comparison

In [93]:
# Advertisement query
ad_query = """
    
SELECT
    fullVisitorId,
    date,
    totals.hits AS hits,
    totals.pageviews AS pageViews,
    totals.timeOnSite AS timeOnSite,
    totals.bounces AS bounces,
    totals.transactions AS transactions,
    totals.totalTransactionRevenue / 1000000 AS revenue,
    channelGrouping,
    
    -- Categorize the sources as 'Google' or 'YouTube'
    CASE 
        WHEN LOWER(trafficSource.source) LIKE '%google%' THEN 'Google'
        WHEN LOWER(trafficSource.source) LIKE '%youtube%' THEN 'YouTube'
        ELSE 'Other'  -- In case there are sources that do not match Google or YouTube
    END AS sourceCategory

FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
    AND (
        LOWER(trafficSource.source) LIKE '%google%' 
        OR LOWER(trafficSource.source) LIKE '%youtube%'
    );

"""    

In [94]:
# Execute the query and store the results in a Pandas DataFrame
ad_df = client.query(ad_query).to_dataframe()

# Display the first few rows
ad_df.head()

,fullVisitorId,date,hits,pageViews,timeOnSite,bounces,transactions,revenue,channelGrouping,sourceCategory
0,1180797935503178227,20170514,1,1,<NA>,1,<NA>,NaN,Organic Search,Google
1,3172363110310924622,20170514,1,1,<NA>,1,<NA>,NaN,Organic Search,Google
2,3043098861438846312,20170514,1,1,<NA>,1,<NA>,NaN,Organic Search,Google
3,0410067780002939622,20170514,1,1,<NA>,1,<NA>,NaN,Organic Search,Google
4,1680957318855673062,20170514,1,1,<NA>,1,<NA>,NaN,Organic Search,Google


In [95]:
# Turn 'date' to datetime object
ad_df['date'] = pd.to_datetime(ad_df['date'], format = "%Y%m%d")

In [96]:
# Save to CSV in a specific directory
ad_df.to_csv("agg_data/ad.csv", index=False)